<a href="https://colab.research.google.com/github/pvn-leo/DataAnalytics/blob/master/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [2]:
cd

/root


In [3]:
cd /content/drive/My Drive/da5

/content/drive/My Drive/da5


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
df=pd.read_csv('./US_accidents1.csv')

In [33]:
print("Rows : ", df.shape[0])
print("Columns : ", df.shape[1])

Rows :  3513501
Columns :  32


In [34]:
df.columns

Index(['Unnamed: 0', 'id', 'source', 'severity', 'start_time', 'end_time',
       'start_lat', 'start_lng', 'description', 'side', 'city', 'state',
       'temperature(f)', 'humidity(%)', 'pressure(in)', 'visibility(mi)',
       'wind_direction', 'wind_speed(mph)', 'weather_condition', 'amenity',
       'bump', 'crossing', 'give_way', 'junction', 'no_exit', 'railway',
       'roundabout', 'station', 'stop', 'traffic_calming', 'traffic_signal',
       'sunrise_sunset'],
      dtype='object')

In [ ]:
'''
df.side.value_counts()
li = list(df['side'].value_counts())
print(len(li))
print(len(df.columns))
minimal_categorical=['side','bump','crossing','give_way','junction','no_exit','railway','roundabout','station','stop','traffic_calming','traffic_signal','sunrise_sunset','year','month','day','hour','weekday']
one_hot_col = []
for i in range(32):
  ok = df.columns[i]
  li = list(df[ok].value_counts())
  if(len(li) >=2 and len(li)<=3):
    one_hot_col.append(ok)

print(one_hot_col)
#['source', 'side', 'amenity', 'bump', 'crossing', 'give_way', 'junction', 'no_exit', 'railway', 'roundabout', 'station', 'stop', 'traffic_calming', 'traffic_signal', 'sunrise_sunset']
'''

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [36]:
da = df.copy()

#Imputations

In [37]:
#weather_condition has null values, hence best method to fill the missing values would be to fill in the mode since, they are categorical values
print(da['weather_condition'].isnull().values.any())
weather_cond_mode = (da['weather_condition'].mode())
print(weather_cond_mode[0])
da["weather_condition"].fillna(weather_cond_mode[0], inplace=True)
print(da['weather_condition'].isnull().sum())
da['weather_condition'].isnull().values.any()

True
Clear
0


False

In [38]:
#wind_direction has null values, hence best method to fill the missing values would be to fill in the mode since, they are categorical values
print(da['wind_direction'].isnull().values.any())
wind_cond_mode = (da['wind_direction'].mode())
print(wind_cond_mode[0])
da["wind_direction"].fillna(wind_cond_mode[0], inplace=True)
print(da['wind_direction'].isnull().sum())
da['wind_direction'].isnull().values.any()

True
Calm
0


False

In [39]:
print(da['temperature(f)'].isnull().values.any())
print(da['humidity(%)'].isnull().values.any())
print(da['pressure(in)'].isnull().values.any())
print(da['visibility(mi)'].isnull().values.any())
print(da['wind_speed(mph)'].isnull().values.any())

True
True
True
True
True


In [40]:
da['temperature(f)'] = da['temperature(f)'].fillna((da['temperature(f)'].mean()))
da['pressure(in)'] = da['pressure(in)'].fillna((da['pressure(in)'].mean()))
da['humidity(%)'] = da['humidity(%)'].fillna((da['humidity(%)'].mean()))
da['visibility(mi)'] = da['visibility(mi)'].fillna((da['visibility(mi)'].median()))
da['wind_speed(mph)'] = da['wind_speed(mph)'].fillna((da['wind_speed(mph)'].median()))

In [41]:
print(da['temperature(f)'].isnull().values.any())
print(da['humidity(%)'].isnull().values.any())
print(da['pressure(in)'].isnull().values.any())
print(da['visibility(mi)'].isnull().values.any())
print(da['wind_speed(mph)'].isnull().values.any())

False
False
False
False
False


# Scaling/Encoding

In [42]:
numerical_cols = ['temperature(f)','humidity(%)','pressure(in)','visibility(mi)','wind_speed(mph)']
#normal distribution - mean is 0 and variance is 1, standard scaling
#scaling
for i in numerical_cols:
    scale = StandardScaler().fit(da[[i]])
    #inplace transformation
    da[i] = scale.transform(da[[i]])

In [43]:
#using pd_getdummies(one hot encoding)
# all of these columns are like binary values true/false, or have atmost 3 unique values
onehotencode = ['side', 'bump', 'crossing', 'give_way', 'junction', 'no_exit', 'railway', 'roundabout', 'station', 'stop', 'traffic_calming', 'traffic_signal', 'sunrise_sunset']
for i in onehotencode:
    da = pd.concat([da,pd.get_dummies(da[i], prefix=i)],axis=1)
    da.drop([i],axis=1, inplace=True)

In [44]:
#label encoding
#many unique values for each attribute, hence label encoding
label_encode=['source','city','weather_condition', 'wind_direction']
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
for i in label_encode:
    da[i] = labelencoder.fit_transform(da[i])
    print(i+" done")

source done
city done
weather_condition done
wind_direction done


In [45]:
da.columns

Index(['Unnamed: 0', 'id', 'source', 'severity', 'start_time', 'end_time',
       'start_lat', 'start_lng', 'description', 'city', 'state',
       'temperature(f)', 'humidity(%)', 'pressure(in)', 'visibility(mi)',
       'wind_direction', 'wind_speed(mph)', 'weather_condition', 'amenity',
       'side_ ', 'side_L', 'side_R', 'bump_False', 'bump_True',
       'crossing_False', 'crossing_True', 'give_way_False', 'give_way_True',
       'junction_False', 'junction_True', 'no_exit_False', 'no_exit_True',
       'railway_False', 'railway_True', 'roundabout_False', 'roundabout_True',
       'station_False', 'station_True', 'stop_False', 'stop_True',
       'traffic_calming_False', 'traffic_calming_True', 'traffic_signal_False',
       'traffic_signal_True', 'sunrise_sunset_Day', 'sunrise_sunset_Night'],
      dtype='object')

# Model

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
target='severity'

unwanted_col=['Unnamed: 0','id', 'start_time','end_time','description','state']
X = da.drop(unwanted_col, axis=1)
y = da[target]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)
logistic_regression = LogisticRegression(random_state=0, verbose=10)
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)
print(acc)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min finished


0.6754323673938133
